## Loading the Dataset & Displaying Structure

In [4]:
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

crema_d_path = "Dataset/CREMA-D_features_dataset.csv"

# Load datasets
dataset = pd.read_csv(crema_d_path)

# drop rows where Emotion is 'unknown'
dataset = dataset[dataset['Emotion'] != 'unknown']


# Display the first few rows of the dataset
print(dataset.head())

# Check the dataset's structure
print(dataset.info())


       RMSE  Zero_Crossing_Rate  Mel_Spectrogram_Mean     MFCC_1      MFCC_2  \
0  0.073580            0.078305            -33.864310 -383.62805   85.888760   
1  0.037229            0.091309            -37.033104 -423.04224  125.774460   
2  0.007056            0.048941            -50.110780 -565.71550  111.819885   
3  0.009229            0.051352            -48.222622 -545.74884  111.447610   
4  0.014069            0.039776            -44.119440 -501.25912  130.523680   

      MFCC_3     MFCC_4     MFCC_5     MFCC_6     MFCC_7  ...  Chroma_11  \
0   1.510415  38.184490  -2.107945  -2.918928 -12.555401  ...   0.373348   
1  12.974215  51.769170 -17.094017  10.942081 -18.850365  ...   0.475053   
2   2.412334  45.476185 -12.045537  21.651108 -13.544354  ...   0.488740   
3  14.475146  46.292442 -11.371361  13.052764 -13.202646  ...   0.424008   
4  15.468277  57.672870  -2.552489  15.085404  -6.516215  ...   0.547534   

   Chroma_12       Tempo    Jitter   Shimmer  Spectral_Flatnes

In [13]:
dataset.shape

(7442, 67)

## Check For Missing Values

In [5]:

# Check for missing values in the dataset
missing_values_summary = dataset.isnull().sum()

# Display columns with missing values, if any
print(missing_values_summary[missing_values_summary > 0])



Series([], dtype: int64)


## Train test split

In [6]:
from sklearn.model_selection import train_test_split

# Separate features and target
X = dataset.drop(columns=['Emotion'])
y = dataset['Emotion']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Display the sizes of the splits
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Testing set size: {X_test.shape[0]} samples")



Training set size: 5953 samples
Testing set size: 1489 samples


##  Handling Class Imbalance

In [7]:
from imblearn.over_sampling import SMOTE

# Ensure y_train is a 1D array for SMOTE
y_train = y_train.squeeze()

# Instantiate SMOTE
smote = SMOTE(random_state=42)

# Apply SMOTE to balance the training set
X_train, y_train = smote.fit_resample(X_train, y_train)

# Verify the new class distribution
print("Class distribution after applying SMOTE:")
print(y_train.value_counts())

# Save the updated training data back to the same variables
print("X_train and y_train have been updated with resampled data.")

print("Class imbalance handled using SMOTE.")


Class distribution after applying SMOTE:
Emotion
neutral    1017
angry      1017
sad        1017
disgust    1017
happy      1017
fearful    1017
Name: count, dtype: int64
X_train and y_train have been updated with resampled data.
Class imbalance handled using SMOTE.


## Normalization of Features

In [8]:
from sklearn.preprocessing import StandardScaler

# Instantiate the scaler
scaler = StandardScaler()

# Fit the scaler on the training data and transform the training data
X_train = scaler.fit_transform(X_train)

# Transform the test data using the same scaler
X_test = scaler.transform(X_test)

# Verify the transformation
print("Training data after normalization:")
print(X_train[:5])

print("Testing data after normalization:")
print(X_test[:5])


Training data after normalization:
[[-5.88578164e-01 -1.48408904e-01 -6.46286071e-01 -5.76589398e-01
  -1.56356651e-01 -4.50431964e-01  1.59694133e-01 -4.30594918e-01
  -1.31203870e-01  1.89324590e-01  8.06197949e-01 -5.00405590e-01
   8.94704041e-01  6.93297206e-01 -6.09452698e-01  9.76456404e-01
  -8.39284563e-01 -6.00010886e-01  8.22568550e-01 -6.97843841e-01
   6.33919281e-01 -2.78078303e-01 -6.68085165e-02 -1.23895192e+00
   5.68608788e-01 -9.71937995e-01  1.74809972e-01  1.98625310e+00
  -3.17720820e-01 -2.10684489e-01 -2.03816951e-01  5.33084629e-01
   1.15716306e+00  5.97765458e-01  9.84865614e-01  2.28100727e-01
  -1.21011535e-01 -1.23846734e+00  1.25221509e-01 -2.16732587e-01
   2.68259689e-01  7.26500221e-01 -8.51106042e-02 -1.50205313e-01
  -3.74111534e-01  5.19696301e-01 -8.75878494e-01 -6.77255191e-01
  -7.89480379e-01 -5.56559005e-01 -3.97649955e-01 -8.02412030e-02
  -6.76117834e-02  5.03152372e-01  9.13801619e-01  2.30947132e-01
  -5.07438588e-01 -4.33899838e-01 -4.5159

## Encode the Target Variable

In [9]:
from sklearn.preprocessing import LabelEncoder

# Instantiate the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the training target
y_train = label_encoder.fit_transform(y_train)

# Transform the testing target using the same encoder
y_test = label_encoder.transform(y_test)

# Check the encoded values
print("Classes:", label_encoder.classes_)
print("Encoded y_train:", y_train[:5])
print("Encoded y_test:", y_test[:5])

print("Target variable encoded successfully!")


Classes: ['angry' 'disgust' 'fearful' 'happy' 'neutral' 'sad']
Encoded y_train: [4 4 0 5 0]
Encoded y_test: [0 3 2 1 1]
Target variable encoded successfully!


## Random Forest Classifier

In [11]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


# Random Forest with default settings
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Test the model
y_pred = rf_model.predict(X_test)
print("Accuracy (Default RF):", accuracy_score(y_test, y_pred))
print("\nClassification Report (Default RF):\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [200, 300],
    'max_depth': [10, 20],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

# Best model evaluation
best_rf_model = grid_search.best_estimator_
y_pred_best = best_rf_model.predict(X_test)
print("Best Parameters (RF):", grid_search.best_params_)
print("Accuracy (Tuned RF):", accuracy_score(y_test, y_pred_best))
print("\nClassification Report (Tuned RF):\n", classification_report(y_test, y_pred_best, target_names=label_encoder.classes_))

# Cross-validation for generalization
cv_scores = cross_val_score(best_rf_model, X_train, y_train, cv=5, scoring='accuracy')
print("Cross-Validation Scores:", cv_scores)
print("Mean CV Accuracy:", cv_scores.mean())

# PCA for dimensionality reduction
pca = PCA(n_components=0.95)  # Retain 95% variance
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Refit Random Forest with PCA-applied data
rf_model_pca = RandomForestClassifier(random_state=42, **grid_search.best_params_)
rf_model_pca.fit(X_train_pca, y_train)
y_pred_pca = rf_model_pca.predict(X_test_pca)
print("Accuracy (PCA RF):", accuracy_score(y_test, y_pred_pca))
print("\nClassification Report (PCA RF):\n", classification_report(y_test, y_pred_pca, target_names=label_encoder.classes_))


Accuracy (Default RF): 0.4613834788448623

Classification Report (Default RF):
               precision    recall  f1-score   support

       angry       0.59      0.70      0.64       254
     disgust       0.44      0.39      0.41       254
     fearful       0.43      0.18      0.25       254
       happy       0.42      0.45      0.43       255
     neutral       0.38      0.49      0.43       218
         sad       0.48      0.56      0.52       254

    accuracy                           0.46      1489
   macro avg       0.46      0.46      0.45      1489
weighted avg       0.46      0.46      0.45      1489

Best Parameters (RF): {'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}
Accuracy (Tuned RF): 0.4714573539288113

Classification Report (Tuned RF):
               precision    recall  f1-score   support

       angry       0.59      0.70      0.64       254
     disgust       0.46      0.42      0.44       254
     fearful       0.41      

## SVM

In [12]:
from sklearn.svm import SVC


# SVM with default settings
svm_model = SVC(random_state=42)
svm_model.fit(X_train, y_train)

# Test the model
y_pred_svm = svm_model.predict(X_test)
print("Accuracy (Default SVM):", accuracy_score(y_test, y_pred_svm))
print("\nClassification Report (Default SVM):\n", classification_report(y_test, y_pred_svm, target_names=label_encoder.classes_))

# Hyperparameter tuning using GridSearchCV
param_grid_svm = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear']
}

grid_search_svm = GridSearchCV(SVC(random_state=42), param_grid_svm, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_svm.fit(X_train, y_train)

# Best model evaluation
best_svm_model = grid_search_svm.best_estimator_
y_pred_best_svm = best_svm_model.predict(X_test)
print("Best Parameters (SVM):", grid_search_svm.best_params_)
print("Accuracy (Tuned SVM):", accuracy_score(y_test, y_pred_best_svm))
print("\nClassification Report (Tuned SVM):\n", classification_report(y_test, y_pred_best_svm, target_names=label_encoder.classes_))

Accuracy (Default SVM): 0.4882471457353929

Classification Report (Default SVM):
               precision    recall  f1-score   support

       angry       0.61      0.68      0.64       254
     disgust       0.47      0.46      0.47       254
     fearful       0.43      0.28      0.34       254
       happy       0.47      0.47      0.47       255
     neutral       0.42      0.50      0.46       218
         sad       0.49      0.54      0.51       254

    accuracy                           0.49      1489
   macro avg       0.48      0.49      0.48      1489
weighted avg       0.48      0.49      0.48      1489

Best Parameters (SVM): {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
Accuracy (Tuned SVM): 0.5030221625251847

Classification Report (Tuned SVM):
               precision    recall  f1-score   support

       angry       0.65      0.69      0.67       254
     disgust       0.49      0.46      0.48       254
     fearful       0.43      0.35      0.38       254
       happy 